In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)

import numpy as np
import json
import pickle 
import h5py
import math
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import random
import soundfile as sf
import os
from pprint import pprint

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0 , 1, 2'
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
from aia_trans import dual_aia_trans_merge_crm
from solver_merge import Solver
from train_utils import parser_all, SEDataset, SEDataLoader, get_noise_clean_path
args = parser_all.parse_args(args = [])
model = dual_aia_trans_merge_crm().cuda()

In [3]:
with open('/workspace/SE_2022/train_noise_by_type.pkl', 'rb') as f:
    train_noise = pickle.load(f)

with open('/workspace/SE_2022/val_noise_by_type.pkl', 'rb') as f:
    val_noise = pickle.load(f)

with open('/workspace/SE_2022/train_map.pkl', 'rb') as f:
    noise_clean_map = pickle.load(f)

data_dir = '/workspace/data/train'
train_path_array = get_noise_clean_path(data_dir, train_noise, noise_clean_map)
val_path_array = get_noise_clean_path(data_dir, val_noise, noise_clean_map)
train_dataset = SEDataset(train_path_array, args.batch_size)
val_dataset = SEDataset(val_path_array, args.batch_size)
train_dataloader = SEDataLoader(data_set=train_dataset,
                                batch_size=1,
                                num_workers=args.num_workers,
                                pin_memory=True)
val_dataloader = SEDataLoader(data_set=val_dataset,
                                batch_size=1,
                                num_workers=args.num_workers,
                                pin_memory=True)
data = {'tr_loader': train_dataloader, 'cv_loader': val_dataloader}

In [4]:
optimizer = torch.optim.Adam(model.parameters(),
                                args.lr,
                                weight_decay=args.l2)
solver = Solver(data, model, optimizer, args)

In [5]:

solver.train()

Begin to train.....
Epoch:1, Iter:0, the average_loss:0.445519, current_loss:0.445519, 1895 ms/batch.
Epoch:1, Iter:1, the average_loss:0.318780, current_loss:0.192041, 1597 ms/batch.
Epoch:1, Iter:2, the average_loss:0.281174, current_loss:0.205961, 1483 ms/batch.
Epoch:1, Iter:3, the average_loss:0.266311, current_loss:0.221722, 1430 ms/batch.
Epoch:1, Iter:4, the average_loss:0.265650, current_loss:0.263008, 1399 ms/batch.
Epoch:1, Iter:5, the average_loss:0.264356, current_loss:0.257886, 1380 ms/batch.
Epoch:1, Iter:6, the average_loss:0.243883, current_loss:0.121040, 1365 ms/batch.
Epoch:1, Iter:7, the average_loss:0.239832, current_loss:0.211475, 1356 ms/batch.
Epoch:1, Iter:8, the average_loss:0.226150, current_loss:0.116698, 1347 ms/batch.
Epoch:1, Iter:9, the average_loss:0.227096, current_loss:0.235613, 1341 ms/batch.
Epoch:1, Iter:10, the average_loss:0.230714, current_loss:0.266894, 1339 ms/batch.
Epoch:1, Iter:11, the average_loss:0.222600, current_loss:0.133346, 1334 ms/b

KeyboardInterrupt: 